# Image Models

## Objectif

Ce notebook permet de générer les jeux de données contenant les pixels des images, puis d'entrainer des modèles.

## 1. Paramètres des jeux de données

Veuillez renseigner les paramètres ci-dessous pour définir les jeux de données

In [ ]:
# Define the dataset size
TRAIN_SIZE = 0.8
TEST_SIZE = 0.2

# Define the images characteristics
WIDTH = 100
HEIGHT = 100
KEEP_RATIO = True
GRAYSCALE = True

# Number of threads to use with images conversion
NB_THREADS = 4

## 2. Génération des jeux de données

Veuillez executer la cellule ci-dessous. Attention, l'étape de pré-processing des images peut être longue si vous avez un gros jeu de données. 

In [ ]:
import os
import pickle
import src.data.data as data
import importlib
from src.data import transformer
from sklearn.model_selection import train_test_split

importlib.reload(transformer)
importlib.reload(data)

df = data.load_data()
target = df["prdtypecode"]
features = df.drop("prdtypecode", axis=1)

X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    train_size=TRAIN_SIZE, 
                                                    test_size=TEST_SIZE)

transformer = transformer.ImagePipeline(width=WIDTH, height=HEIGHT, 
                                     keep_ratio=KEEP_RATIO, 
                                     grayscale=GRAYSCALE,
                                     nb_threads=NB_THREADS)

img_output_dir = transformer.output_dir.split('/')[1]
pipeline = transformer.pipeline

print("Transformation du jeu de données d'entrainement")
X_train = pipeline.fit_transform(X_train)
print("Transformation du jeu de données de test")
X_test = pipeline.transform(X_test)

files = {
    "data/training/X_train.pkl": X_train,
    "data/training/X_test.pkl": X_test,
    "data/training/y_train.pkl": y_train,
    "data/training/y_test.pkl": y_test
}

print("Enregistrement des données")
if not os.path.exists("data/training"):
    os.makedirs("data/training")

for filename in files.keys():
    with open(filename,'wb') as fp:
            pickle.dump(files[filename], fp)

Attention, il est maintenant recommandé de redémarrer le kernel Jupyter pour vider la RAM utilisée par la cellule ci-dessus.

## 3. Entrainement du modèle

Instanciez votre modèle dans la cellule ci-dessous, puis executer les cellules à la suite.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

ESTIMATOR_FILE_NAME = "KNeighborsClassifier_n_neighbors-7"
ESTIMATOR = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)

In [ ]:
from sklearn.metrics import accuracy_score
import os
import pickle

X_train = pickle.load(open("data/training/X_train.pkl", "rb"))
y_train = pickle.load(open("data/training/y_train.pkl", "rb"))

X_test = pickle.load(open("data/training/X_test.pkl", "rb"))
y_test = pickle.load(open("data/training/y_test.pkl", "rb"))

ESTIMATOR.fit(X_train, y_train)
y_train_pred = ESTIMATOR.predict(X_train)
print("Score train :", accuracy_score(y_true=y_train, y_pred=y_train_pred))
y_test_pred = ESTIMATOR.predict(X_test)
print("Score test  :", accuracy_score(y_true=y_test, y_pred=y_test_pred))

if not os.path.exists("data/models"):
    os.makedirs("data/models")

with open(f"data/models/{ESTIMATOR_FILE_NAME}",'wb') as fp:
    pickle.dump(ESTIMATOR, fp)